# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pl.read_csv(demonstration_fp, try_parse_dates=True, dtypes={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical})
df.head()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow…","""male""",22,0,7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. …","""female""",38,0,71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis…","""female""",26,0,7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs.…","""female""",35,0,53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil…","""male""",35,0,8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': Int64,
 'Name': Utf8,
 'Sex': Categorical,
 'Age': Int64,
 'Parch': Int64,
 'Fare': Float64,
 'Cabin': Utf8,
 'Embarked': Categorical,
 'Birthday': Date,
 'Board time': Time,
 'Married since': Datetime(time_unit='us', time_zone=None),
 'all_NA': Utf8}

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [5]:
df["Birthday"][0]

datetime.date(1937, 10, 28)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [6]:
meta_dataset = MetaDataset.from_dataframe(df)

/Users/qubix/Documents/work/metasynth/metasynth/metasynth/provider.py:327: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [7]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'Utf8', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'Categorical', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.multinoulli', 'provenance': 'builtin', 'class_name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'p

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [8]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [9]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
174,"""e%)QBL^\@ur]c!…","""male""",28,0,20.357411,"""G:v+-??`%""","""C""",1939-12-22,14:53:00,2022-08-01 02:22:02,null
363,"""7PnO)LX#N*rlw?…","""male""",2,0,9.026833,null,"""C""",1925-01-15,17:17:54,2022-08-03 15:52:15,null
23,"""cQjZ?PlTUR&@E\…","""male""",72,0,31.096885,null,"""S""",1939-05-03,11:47:06,2022-07-21 10:59:11,null
264,"""@!hn&9<Y']g1T6…","""female""",null,0,14.800279,"""T1[r1D]F.""","""C""",1926-04-06,16:20:32,null,null
47,"""eTSD8Yu^l1qNu&…","""male""",59,0,86.72177,null,"""C""",1923-03-24,12:15:11,2022-08-05 21:47:16,null


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [10]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'Int64',
 'prop_missing': 0.0,
 'distribution': {'implements': 'core.unique_key',
  'provenance': 'builtin',
  'class_name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [11]:
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,f32,cat,date,time,datetime[μs],f32
1,"""!D_TuA2'&43""YP…","""female""",null,1,12.44073,null,"""S""",1916-06-20,12:59:48,2022-07-25 06:14:01,null
2,"""@4bWY FpY OB …","""female""",32,1,38.941822,null,"""S""",1933-10-18,16:59:36,null,null
3,""" DCE %bd;8V`S^…","""male""",27,1,23.67861,null,"""C""",1919-11-01,11:26:50,2022-08-08 13:01:31,null
4,"""5G?+ C~Ar:oh""","""male""",11,0,82.35974,null,"""S""",1936-02-14,14:17:42,2022-07-30 06:38:18,null
5,"""*q LtbLu}HoA U…","""female""",null,0,30.071594,null,"""Q""",1932-08-31,18:29:31,2022-07-29 17:47:13,null


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [12]:
# First, we create a specification dictionary for the variables
from metasynth.distribution import FakerDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,f32,cat,date,time,datetime[μs],f32
1,"""Kathryn Tran""","""male""",11,0,66.049675,null,"""S""",1913-11-20,17:59:17,2022-07-28 01:11:57,null
2,"""Eric Allen""","""male""",null,0,32.81577,null,"""S""",null,11:40:09,2022-07-31 09:02:21,null
3,"""Cory Martinez""","""male""",null,2,24.335832,null,"""S""",1924-12-12,13:26:26,null,null
4,"""Kimberly Chave…","""female""",26,0,35.041956,null,"""Q""",1927-08-30,15:16:07,2022-07-22 22:13:43,null
5,"""Laura Moore""","""female""",76,0,21.441304,null,"""S""",1909-06-13,16:24:49,2022-08-08 05:38:20,null


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [13]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Kathryn Tran""","""male""",null,0,0.783199,null,"""S""",1927-12-16,15:36:57,2022-07-31 12:40:58,null
2,"""Eric Allen""","""female""",36,0,2.750131,null,"""S""",1924-06-27,14:53:16,2022-07-20 02:24:42,null
3,"""Cory Martinez""","""male""",26,0,0.363102,"""F)L@ 6]_,""","""S""",null,16:29:13,2022-08-11 00:45:53,null
4,"""Kimberly Chave…","""female""",null,1,0.668443,"""FCnyH1N$""","""S""",null,13:51:05,2022-08-02 06:48:29,null
5,"""Laura Moore""","""male""",22,0,0.076071,null,"""Q""",1911-08-14,16:33:26,2022-08-11 23:54:38,null


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [14]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,f32,cat,date,time,datetime[μs],f32
1,"""Kathryn Tran""","""male""",23,0,5.737904,null,"""C""",1931-01-16,17:14:07,2022-07-30 08:47:16,null
2,"""Eric Allen""","""male""",null,0,2.506943,null,"""S""",1909-04-20,18:12:06,2022-08-13 02:48:36,null
3,"""Cory Martinez""","""male""",null,0,0.522842,null,"""C""",1907-05-26,15:37:13,2022-07-19 20:34:13,null
4,"""Kimberly Chave…","""male""",36,1,0.469086,null,"""S""",1932-05-13,17:54:06,2022-08-08 08:25:55,null
5,"""Laura Moore""","""male""",27,1,0.326631,null,"""Q""",1906-09-30,null,2022-08-12 01:06:05,null
6,"""Jose Collins""","""male""",null,0,0.177057,null,"""S""",1906-07-26,17:23:07,2022-08-06 20:41:11,null
7,"""Sara Smith""","""male""",null,0,0.432775,null,"""S""",1928-09-12,15:29:49,2022-07-19 02:16:47,null
8,"""George Hood""","""male""",null,1,1.454606,null,"""S""",1932-07-15,17:44:45,2022-08-03 13:00:43,null
9,"""Makayla Oneal""","""male""",34,0,0.359105,null,"""S""",1911-02-21,10:44:05,2022-08-04 00:50:53,null
